In [ ]:
import numpy as np 
from scipy.interpolate import RegularGridInterpolator
from scipy.integrate import solve_ivp

def integrate_particles(positions, concentrations, velocity_field, X, Y, dt, num_steps, L):
    #positions is of the form np.ones((6, 2)), concentrations of the form np.ones((6, 1))
    num_particles = len(positions)
    positions_over_time = np.zeros((num_steps+1, num_particles, 2))  #positions_over_time[0] gives positions at t=0
    concentrations_over_time = np.zeros((num_steps+1, num_particles, 1))
    positions_over_time[0] = positions
    concentrations_over_time[0] = concentrations

    u, v = get_matrix_u(velocity_field), get_matrix_v(velocity_field)
    polynomial_u, polynomial_v = interpolate(u), interpolate(v)

    t_span = (0, num_steps * dt)
    t_eval = [i * dt for i in range(num_steps+1)]
    #y0 = [ [positions[i][0], positions[i][1]] for i in range(num_particles)]
    t = [i*dt for i in range(num_steps+1)]
    y0 = positions
    vector_rk4(fun, y0, t)

    return positions_over_time


def get_matrix_u(velocity_field):
    umatrix = velocity_field
    nt, nx, ny = velocity_field.shape
    for i in range(nt):
        umatrix[i] = np.gradient(velocity_field[i], axis=1) * -1
    return umatrix

def get_matrix_v(velocity_field):
    vmatrix = velocity_field
    nt, nx, ny = velocity_field.shape
    for i in range(nt):
        vmatrix[i] = np.gradient(velocity_field[i], axis=0) 
    return vmatrix

def interpolate(umatrix):

    t = umatrix.shape[0]
    time = [i for i in range(t)]

    interpolator = RegularGridInterpolator((time, X, Y), umatrix, method = "cubic")

    return interpolator

def fun(t, y):
    #note that here y refers to both x and y coords
    xlist = [sublist[0] for sublist in y] + L
    ylist = [sublist[1] for sublist in y] + L
    xlist = xlist % 2*L
    ylist = ylist % 2*L
    xlist -= L
    ylist -= L

    coords = np.array([t*np.ones(num_particles), np.array(xlist).reshape(-1, 1), np.array(ylist).reshape(-1, 1)])
    #make mod L

    dxdt = polynomial_u(coords) 
    dydt = polynomial_v(coords)
    dxdt=np.array(dxdt).reshape(-1, 1)
    dydt=np.array(dydt).reshape(-1, 1)
 

    result = np.concatenate((dxdt, dydt), axis=1)


    return result

def vector_rk4(f, y0, t):
    #y0 will be of shape num_particles(N) x 2, same for all k values 
    #t = [dt, 2dt, 3dt,...]
    #f takes a scalar t[i] and a N x 2 array
    n = len(t)
    #y = np.zeros((n, len(y0))) #this is positions_over_time
    y = positions_over_time
    #y[0] = y0 #we have already set the firstvalue
    for i in range(n - 1):
        h = t[i+1] - t[i]
        k1 = f(t[i], y[i])
        k2 = f(t[i] + 0.5*h, y[i] + 0.5*h*k1)
        k3 = f(t[i] + 0.5*h, y[i] + 0.5*h*k2)
        k4 = f(t[i+1], y[i] + h*k3)
        y[i+1] = y[i] + (h/6)*(k1 + 2*k2 + 2*k3 + k4)
    return y